<a href="https://www.kaggle.com/code/dhirajmishra57/udemy-pyspark-tutorial?scriptVersionId=134135701" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Pyspark End-To-End

`Course - https://www.udemy.com/course/pyspark-end-to-end-developer-course-spark-with-python/learn/lecture/37180040#overview`

## Topics -
    - How to read flat file and tables using jbdc connection
    - DataFrame API Selection
    - DataFrame Where & Filter
    - DataFrame API Sorting
    - DataFrame API Set
    - DataFrame API Join
    - DataFrame API Windowing
    - DataFrame API Aggregation
    - DataFrame API GroupBy
    - DataFrame APIs Null Functions

## Read & Write Files
    - CSV
    - JSON
    - Parquet
    - XML, Avro, Binary

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=0d3391f1902f4a42fdc3a0ec0bce484f9ed1a3bb09dac1031124a6976cd7a43c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [4]:
import pyspark
from pyspark.sql import SparkSession,functions
from pyspark.sql.functions import *
from pyspark.sql.types import StructField,StructType,IntegerType,StringType
from pyspark.sql import window
from pyspark.sql.window import *

spark = SparkSession.builder.appName("Pyspark-Tutorial").getOrCreate()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 14:00:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/kaggle/input/retail-db-tutorial/RetailDB SalesData/Customers/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Orders/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Products/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Order_items/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Categories/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Departments/part-00000


## How to read file using various techniques

In [ ]:
ord_file_path = "/kaggle/input/retail-db-tutorial/RetailDB SalesData/Orders"
ord = spark.read.load(ord_file_path,sep=",",format='csv',schema=('ordid int,ord_date timestamp,ord_cust_id int, ord_status string'))
ord.show(1)

ord = spark.read.csv(ord_file_path,sep=",",schema=('ordid int,ord_date timestamp,ord_cust_id int, ord_status string'))
ord.show(1)

ord = spark.read.format('csv').load(ord_file_path,sep=',',schema='ordid int,ord_date timestamp,ord_cust_id int, ord_status string')
ord.show(1)

## How to read table and jdbc connection

In [ ]:
"""
spark.read.jdbc("connection string / jdbc:mysql://localhost:3306/emp","table",
                properties={"user":"root","password":"root","driver":"com.mysql.cj.jdbc.Driver"})

spark.read.format("jdbc") \
.option("url","connection string") \
.option("driver","com.mysql.cj.jdbc.Driver") \
.option("user","root")
.option("password":"root")
.option("dbtable","employee")
# .option("dbtable","select empid, salary from employee where gender = 'M' ")
# .option("query"," select empid, salary from employee where gender = 'M' ")
.load()

"""
print("Hello")

# DataFrame API Selection

### Select
Select one or more columns
Applying necessary functions on select

### SelectExpr
This is a variant of select that accepts SQL expressions.
If we want to use any functions available in SQL but not in Spark Built-in functions, then we can use selectExpr
            -- explode

### withColumn(colName, col)
Applied transformation to only selected columns.
The first argument is an alias name. If we give an alias name same as a column name, the transformations will apply
on the same column.
Otherwise a new column will be formed. Avoid giving alias name same as column name.
ord.withColumn('order_month',substring(orderDF.order_date,1,10)).show()


### withColumnRenamed(existingCol, newCol)
Rename Existing Column.
ord.withColumnRenamed('order_id','order_id1').show()

### drop(*cols)
Drop a column.
order = ord.drop('order_id','order_date')

### dropDuplicates(subset=None)
Drop duplicate rows.
Optionally can consider only subset of columns.
emp.dropDuplicates().show()
emp.dropDuplicates(("name","score1","score2")).show()

In [ ]:
ord
data = [('Robert',35,40,40),('Robert',35,40,40),('Ram',31,33,29),('Ram',31,33,91)]
emp = spark.createDataFrame(data=data,schema=['name','score1','score2','score3'])

In [ ]:
ord.select("ordid",'ordid',ord.ordid).show(5)

ord.select(lower("ord_status").alias("lower_sts"),"ord_status").show(5)

In [ ]:
# ord.show(5)
ord.selectExpr("substring(ord_date,1,4) as ord_year","ord_date") \
.selectExpr("ord_year","substring(ord_date,6,2) as ord_month","ord_date") \
.selectExpr("ord_date","ord_month","ord_year","substring(ord_date,9,2) as ord_day") \
.select("ord_year","ord_month","ord_date") \
.show(5)

In [ ]:
ord.withColumn("ord_year",substring("ord_date",1,4)).show(5)

ord.withColumn("ord_date",substring("ord_date",1,4)).show(5)

In [ ]:
ord.withColumnRenamed("ord_status","ord_sts").show(5)

In [ ]:
ord.drop('ord_status','ord_date').show(2)
ord.drop('ord_status').show(2)

In [ ]:
emp.show()

emp.dropDuplicates().show() ## only 1 record filtered

emp.dropDuplicates(["name"]).show()

emp.dropDuplicates(["name","score2"]).show()

# DataFrame API Filter

### filter(condition): (Its alias ‘where’)
Filter rows using a given condition

use '&' for 'and‘. '|' for 'or‘. (boolean expressions)

Use column function isin() for multiple search Or use IN Operator for SQL Style syntax.

In [ ]:
# &
ord.where((ord.ordid > 10) & (ord.ordid < 20)).show(5)

ord.filter((ord.ordid > 10) & (ord.ordid < 20)).show(5)

# |
ord.where((ord.ordid == 10) | (ord.ordid == 20)).show(5)

ord.where((ord.ordid == 10) | (ord.ordid == 20)).show(5)

# isin
ord.where(ord.ord_status.isin("CLOSED","PENDING")).show(5)
ord.filter(ord.ord_status.isin("CLOSED","PENDING")).show(5)

# sql like where optional
ord.where("ord_status in ('CLOSED','NOT-PENDING') ").show(5)

# DataFrame APIs Sort


### sort() or orderBy()
orderBy() is an alias for .sort()
Sort specific column(s).

### sortWithinPartitions
At time, we may not want sort globally, but with in a group. In that case we can use sortWithinPartitions.

In [ ]:
ord.sort(ord.ord_status.desc()).show(2)
ord.sort(col("ord_status").desc()).show(2)
ord.sort("ord_status","ord_cust_id").show(2)
ord.sort("ord_status","ord_cust_id",ascending=[0,1]).show(2)

In [ ]:
data = [("a",1),("b",2),("c",3),("d",4),("e",5)]
df = spark.createDataFrame(data,schema='col1 string,col2 int')
df = df.repartition(2)
df = df.withColumn("partid",spark_partition_id())

df.sort(df.col1).show()
df.sortWithinPartitions(df.col1).show()

## DataFrame APIs : Set Operators

### union() and unionAll():
Same and contains duplicate values
Use distinct after union or unionAll to remove duplicates

### unionByName():
The difference between this function and :func:`union` is that this function
resolves columns by name (not by position)

### intersect(): 
Containing rows in both DataFrames. Removed duplicates

### intersectAll():
Same as intersect. But retains the duplicates

### exceptAll():
Rows present in one DataFrame but not in another

In [ ]:
df1 = spark.range(4)
df2 = spark.range(3,5)

# Union and UnionAll performs same way

df1.union(df2).show()
df1.unionAll(df2).show()

In [ ]:
df1.intersect(df2).show()
df1.intersectAll(df2).show()

# DataFrame APIs : Join

Join Type
inner INNER JOIN, JOIN
outer, full, fullouter,
full_outer
FULL OUTER JOIN
left,
left_outer,leftouter
LEFT JOIN
right,right_outer,rig
htouter
RIGHT JOIN
cross CROSS JOIN
left_anti, leftanti
leftsemi,left_semi

In [ ]:
df1 = spark.createDataFrame(data=[(1,'Robert'),(2,'Ria'),(3,'James')],schema='empid int, empname string')
df2 = spark.createDataFrame(data=[(2,'USA'),(4,'India')],schema='empid int, country string')

# Inner Join
df1.join(df2,df1.empid == df2.empid).select(df1.empid.alias("employeeid"),df1.empname,df2.country).show()

# left anti join
# cannot get right table attributes and it executes faster as compared to inner as it doesn't check right/second table attributes
df1.join(df2,df1.empid == df2.empid,"leftanti").show()

# Left outer or Left
df1.join(df2,df1.empid == df2.empid,"left")\
.select(df1.empid,df1.empname,df2.country) \
.sort(df2.country.desc()) \
.show()

# full outer
df1.join(df2,df1.empid == df2.empid,"full")\
.select(df1.empid,df1.empname,df2.country) \
.sort(df1.empid.asc()) \
.show()

In [ ]:
# self

df1 = spark.createDataFrame(data=((1,'Robert',2),(2,'Ria',3),(3,'James',5)),schema='empid int,empname string,managerid int')
df1.show()
df1.alias("emp1").join(df1.alias("emp2"),col("emp1.managerid") == col("emp2.empid"))\
.select(col("emp1.empid").alias("EmployeeID"),col("emp1.empname").alias("EmployeeName"),col("emp2.empid").alias("ManagerID"),col("emp2.empname").alias("ManagerName")) \
.show()

### Window Functions

Lets see 3 types of Window Functions:
Ranking
Analytical
Aggregate

In [ ]:
data = (("James","Sales","NY",9000,34),
("Alicia","Sales","NY",8600,56),
("Robert","Sales","CA",8100,30),
("John","Sales","AZ",8600,31),
("Ross","Sales","AZ",8100,33),
("Kathy","Sales","AZ",1000,39),
("Lisa","Finance","CA",9000,24),
("Deja","Finance","CA",9900,40),
("Sugie","Finance","NY",8300,36),
("Ram","Finance","NY",7900,53),
("Satya","Finance","AZ",8200,53),
("Kyle","Marketing","CA",8000,25),
("Reid","Marketing","NY",9100,50)
)

schema=("empname","dept","state","salary","age")

df = spark.createDataFrame(data=data,schema=schema)

In [ ]:
spec = Window.partitionBy("dept").orderBy(col("salary").desc())

df.withColumn("rank",rank().over(spec))\
.withColumn("drank",dense_rank().over(spec))\
.withColumn("rownum",row_number().over(spec))\
.withColumn("perrnk",percent_rank().over(spec))\
.withColumn("perrnk",percent_rank().over(spec))\
.show()

In [ ]:
spec = Window.partitionBy("dept").orderBy(col("salary").desc())

df.select("dept","salary")\
.withColumn("lead_salary",lead("salary",1,-1).over(spec))\
.withColumn("lag_salary",lag("salary",1,-1).over(spec))\
.show()

In [ ]:
spec_without_orderby = Window.partitionBy("dept")
spec_with_orderby = Window.partitionBy("dept").orderBy(col("salary").asc()) ## rolling sum if we use order by

df.select("dept","salary")\
.withColumn("sum_slry_without_orderby",sum("salary").over(spec_without_orderby))\
.withColumn("sum_slry_with_orderby",sum("salary").over(spec_with_orderby))\
.withColumn("avg_slry_without_orderby",avg("salary").over(spec_without_orderby))\
.withColumn("avg_slry_with_orderby",avg("salary").over(spec_with_orderby))\
.withColumn("min_salary",min("salary").over(spec_without_orderby))\
.withColumn("max_salary",max("salary").over(spec_without_orderby))\
.withColumn("count_salary",count("salary").over(spec_without_orderby))\
.show()

### Window Functions

* rangeBetween:
    - Takes two argument (start,end) to define frame boundaries.
    - Default : unboundedPreceding and unboundedFollowing

* rowsBetween:
    - Takes two argument (start,end) to define frame boundaries.
    - Deafult : unboundedPreceding and unboundedFollowing.

In [ ]:
spec1=Window.partitionBy(df.dept).orderBy(df.salary).rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df.select(df.dept,df.salary).withColumn("sum_sal",sum("salary").over(spec1)).show()
spec1=Window.partitionBy(df.dept).orderBy(df.salary).rangeBetween(Window.currentRow, Window.unboundedFollowing)
df.select(df.dept,df.salary).withColumn("sum_sal",sum("salary").over(spec1)).show()
spec1=Window.partitionBy(df.dept).orderBy(df.salary).rangeBetween(Window.currentRow,500)
df.select(df.dept,df.salary).withColumn("sum_sal",sum("salary").over(spec1)).show()

# DataFrame APIs : Aggregations

    - Summary
    - avg, min, max, sum, sumDistinct, count, countDistinct -- all can be used in same/single `select`
    - df = spark.createDataFrame(((1,100),(2,150),(3,200),(4,50),(5,50)),schema='id int,salary int')
    - df.select(collect_list(df.salary)).show(truncate=False)


In [ ]:
ord_items = "/kaggle/input/retail-db-tutorial/RetailDB SalesData/Order_items/part-00000"

orditems = spark.read.format("csv").load(ord_items,sep=",",schema='order_item_id int,ord_item_order_id int, order_item_product_id int, quantity int,subtotal float, price float')
orditems.summary().show()

In [ ]:
orditems.select("price").summary().show()

In [ ]:
orditems.select(avg("price").alias("avg_price"),min("price").alias("min_price"),max("price").alias("max_price")).show()

In [ ]:
orditems.select(avg("price").alias("avg_price"),
                min("price").alias("min_price"),
                max("price").alias("max_price"),
                sum("price").alias("sum_price"),
                sumDistinct("price").alias("sum_distinct_price")
               ).show()

In [ ]:
orditems.select(avg("price").alias("avg_price"),
                min("price").alias("min_price"),
                max("price").alias("max_price"),
                sum("price").alias("sum_price"),
                sumDistinct("price").alias("sum_distinct_price"),
                count("price").alias("count_price"),
                countDistinct("price").alias("distinct_price_count")
               ).show()

In [ ]:
df = spark.createDataFrame(((1,100),(2,150),(3,200),(4,50),(5,50)),schema='id int,salary int')
df.show()
df.select(collect_list(df.salary)).show(truncate=False) ## shows duplicate value as well
df.select(collect_set(df.salary)).show(truncate=False)  ## Only shows unique values

# DataFrame API GroupBy

In [ ]:
data = (("James","Sales","NY",9000,34),
("Alicia","Sales","NY",8600,56),
("Robert","Sales","CA",8100,30),
("Lisa","Finance","CA",9000,24),
("Deja","Finance","CA",9900,40),
("Sugie","Finance","NY",8300,36),
("Ram","Finance","NY",7900,53),
("Kyle","Marketing","CA",8000,25),
("Reid","Marketing","NY",9100,50))

schema=("empname","dept","state","salary","age")
df = spark.createDataFrame(data=data,schema=schema)
df.show()

### Filtering records before performing aggregations

In [ ]:
groupped_data = df.where(df.state == "NY").groupBy("dept")
groupped_data.agg(min("salary").alias("min_salary"),max("salary").alias("max_salary"),avg("salary").alias("avg_salary")).show()

### Filtering records after performing aggregations

In [ ]:
groupped_data = df.groupBy("dept")
groupped_data.agg(min("salary").alias("min_salary"),
                  max("salary").alias("max_salary"),
                  avg("salary").alias("avg_salary"))\
.where(col("min_salary") >= 8000)\
.show()

### Without Filters

In [ ]:
groupped_data = df.groupBy("dept")
groupped_data.agg(min("salary").alias("min_salary"),
                  max("salary").alias("max_salary"),
                  avg("salary").alias("avg_salary"))\
.show()

# DataFrame APIs : Null Functions

In [ ]:
# Prepare Data
df = spark.createDataFrame((('Robert',1, None,114.0), ('John',None, 2577,float('nan'))), ("name", "id","phone","stAdd"))
df.show()

In [ ]:
# To check Null values
df.select("phone",isnull("phone").alias("nullcheck")).show()

## Advance filtering based on true false
df.select("phone",isnull("phone").alias("nullcheck")).filter(col("nullcheck")).show()

In [ ]:
# To check NaN == Not a Number values
df.select("stAdd",isnan("stAdd").alias("nullcheck")).show()

## Advance filtering based on true false
df.select("stAdd",isnan("stAdd").alias("nullcheck")).filter(col("nullcheck")).show()

In [ ]:
# Returns col1 if it is not NaN, or col2 if col1 is NaN.
df.select(df.stAdd,df.phone,nanvl(df.stAdd,df.phone)).show()

## Very Important

##### Drop can drop any rows and columns

In [ ]:
## Drop can drop columns and rows
data=(('Alice',80,10),('Bob',None,5),('Tom',50,50),(None,None,None),('Robert',30,35))
schema='name string, age int, height int'
df = spark.createDataFrame(data,schema)
df.show()
df.na.drop().show()
df.where(~isnull("age")).drop().show()
df.drop(df.age).show()

In [ ]:
df.na.fill(-1).show()

In [ ]:
### emp2 DataFrame
data=(('Robert',35,40,40),('Ram',31,33,29),('John',95,89,91))
schema = ('name','score1','score2','score3')
emp2= spark.createDataFrame(data=data, schema=schema)

emp2.select(emp2.score1, emp2.score2,sequence(emp2.score1,emp2.score2).alias('new_col')).show(truncate=False)

# Read & Write Files


    - CSV
    - JSON
    - Parquet
    - XML, Avro, Binary

## Reading CSV

    - Read multiple CSV files
    - Read all CSV files in a directory
            ``spark.read.csv("path") & dataframe.write.csv("path")``
    - Options while reading CSV file
            * delimiter
            * InferSchema
            * header
            * quotes
            * nullValues
            * dateFormat
    - Write DataFrame to CSV file
            * Using options
            * Saving Mode

In [ ]:
spark.read.csv(path1).option("header",True)
spark.read.csv("path1,path2,path3").option("header",True).option()
spark.read.csv(Folderpath).option("header",True).option()